In [3]:
from datetime import datetime
    
def convert_timestring_to_seconds(time_string):
    '''Convert time_string to Unix time: timestamp()'''

    '''It turns out the time format the *.csv file is all \dd-mm-yyyy hh:mm\ format'''

    # Using datetime.strptime()
    if time_string.find('/') != -1:
        print('find slash character')
        dt = datetime.strptime(time_string, "%m/%d/%Y %H:%M")
    else: # dd-mm-yyyy H:M
        '''time_string = time_string.replace('-', '/')
        print('time_string:', time_string)
        loc1 = time_string.find('/')
        loc2 = time_string.rfind('/')
        dd = time_string[0:loc1]
        print('dd', dd)
        mm = time_string[loc1+1:loc2]
        print('mm:', mm)
        result_string = mm +'/'+ dd +time_string[loc2:]
        print('result_string:', result_string)
        dt = datetime.strptime(result_string, "%m/%d/%Y %H:%M")'''

        dt = datetime.strptime(time_string, "%d-%m-%Y %H:%M") # Note: Y upper case is necessary

    Unix_time = dt.timestamp()
    return Unix_time

def convert_TwoTimeFields_to_seconds(date_field, hhmmss_am_or_pm_string):
    '''Convert time_string to Unix time: timestamp()'''
    #print('type(date_field):', type(date_field))
    #print('type(hhmmss_am_or_pm_string):', type(hhmmss_am_or_pm_string))
    #print(date_field + hhmmss_am_or_pm_string)

    date_string = date_field.strftime('%Y-%m-%d') # convert from timestamp format to str 
    datetimeObj = datetime.strptime(date_string +' '+ hhmmss_am_or_pm_string, "%Y-%m-%d %I:%M:%S %p") # I, not H

    Unix_time = datetimeObj.timestamp()
    return Unix_time

In [4]:
import pandas as pd
import os

## the following way is wasteful and error prone upon \\\\ and \/\ char in comments in the file
#df_DataHistorian_ServiceNow = pd.read_csv('SNOW CSV - IndyPar Historian.csv'
#df_DataHistorian_ServiceNow = pd.read_csv('SNOW CSV - IndyPar Historian.csv', encoding = 'unicode_escape', engine ='python')

# read columns of data that of interest
fields = ["assigned_to", "work_start", "work_end"]
#df_DataHistorian_ServiceNow = pd.read_csv('SNOW CSV - IndyPar Historian.csv', skipinitialspace=True, usecols=fields)
df_DataHistorian_ServiceNow = pd.read_csv('SNOW CSV - IndyPar Historian.csv', usecols=fields)

#print(df_DataHistorian_ServiceNow)
#print(df_DataHistorian_ServiceNow['assigned_to'][0])
#print(df_DataHistorian_ServiceNow['work_start'][3])
#print(df_DataHistorian_ServiceNow['work_end'][0])

nrows_DataHistorian_ServiceNow = df_DataHistorian_ServiceNow.shape[0]
print(nrows_DataHistorian_ServiceNow)

dataFrame1 = pd.DataFrame(columns = ['System ID', 'start_time', 'end_time'])

for i in range(nrows_DataHistorian_ServiceNow):
    strg = df_DataHistorian_ServiceNow['assigned_to'][i] # extract ID from 'assigned to'
    #print('strg type is:', type(strg))
    if( not (pd.isna(strg))):
        pos = strg.find('(')
        #df_DataHistorian_ServiceNow['assigned_to'][i] = strg[pos+1:-1]
        dataFrame1.loc[i, 'System ID'] = strg[pos+1:-1]
    #print(df_DataHistorian_ServiceNow['work_start'][i])
    #print(df_DataHistorian_ServiceNow['work_end'][i])

    if( not (pd.isna(df_DataHistorian_ServiceNow['work_start'][i]))):
        dataFrame1.loc[i, 'start_time'] = convert_timestring_to_seconds(df_DataHistorian_ServiceNow['work_start'][i])
    else:
        dataFrame1.loc[i, 'start_time'] = 0.0    
    if( not (pd.isna(df_DataHistorian_ServiceNow['work_end'][i]))):   
        dataFrame1.loc[i, 'end_time'] = convert_timestring_to_seconds(df_DataHistorian_ServiceNow['work_end'][i])
    else:
        dataFrame1.loc[i, 'end_time'] = 0.0    

dataFrame1

317


,System ID,start_time,end_time
0,C027377,1.58213e+09,1.58213e+09
1,GA82374,0,0
2,U3X7569,1.61947e+09,1.61947e+09
3,NaN,0,0
4,C027377,1.62156e+09,1.62156e+09
...,...,...,...
312,C027377,1.59837e+09,1.59837e+09
313,EPX7048,1.63631e+09,1.63631e+09
314,L008896,0,0
315,C027377,1.63638e+09,1.63639e+09


In [19]:
df_DataHistorian_AuditTrail= pd.read_excel(
     os.path.join("Example_AuditTrail_DH.xlsm"),
     sheet_name ='Audits', usecols=[0, 2],
     engine='openpyxl',
)

#print(df_DataHistorian_AuditTrail)
print(df_DataHistorian_AuditTrail.iloc[0:922, :])

#print(df_DataHistorian_AuditTrail['Username'])
#print(df_DataHistorian_AuditTrail['Occurred'])
#print(df_DataHistorian_AuditTrail['Time'])

# extract ID from 'assigned to'
nrows_DataHistorian_AuditTrail = df_DataHistorian_AuditTrail.shape[0]
print(nrows_DataHistorian_AuditTrail)


dataFrame2 = pd.DataFrame(columns = ['System ID', 'TimeInUnixTime', 'TimeString'])

rowStartValidData = 920
for i in range(rowStartValidData, nrows_DataHistorian_AuditTrail):
#for i in range(rowStartValidData, 1000):    # temporary
    strg = df_DataHistorian_AuditTrail.loc[i]
    #print('strg:', strg)
    #dataFrame2.loc[i, 'Time'] = strg[0] 
    dataFrame2.loc[i, 'System ID'] = strg[1]
    #print(df_DataHistorian_AuditTrail['Occurred'][i])
    #print(df_DataHistorian_AuditTrail['Time'][i])    

    datetimeObj = strg[0]
    dataFrame2.loc[i-rowStartValidData, 'TimeInUnixTime'] = datetimeObj.timestamp()
    dataFrame2.loc[i-rowStartValidData, 'TimeString'] = datetimeObj
    #print('i-rowStartValidData = ', i-rowStartValidData)

#dataFrame2.loc[920:1000] 
#print('---------------------')
#dataFrame2.iloc[100]\n


           Search Start:  Unnamed: 2
0            Search End:         NaN
1     Script Completion:         NaN
2        Timestamp_Local      UserID
3    2021-07-01 00:00:00  IPODHAdmin
4    2021-07-01 00:00:00  IPODHAdmin
..                   ...         ...
917  2021-07-05 16:00:00  IPODHAdmin
918  2021-07-05 16:00:00  IPODHAdmin
919  2021-07-05 16:00:01  IPODHAdmin
920  2021-07-05 16:04:13  c027377-ds
921  2021-07-05 16:17:59  c027377-ds

[922 rows x 2 columns]
36401


"\ndataFrame2 = pd.DataFrame(columns = ['System ID', 'TimeInUnixTime', 'TimeString'])\n\nrowStartValidData = 920\nfor i in range(rowStartValidData, nrows_DataHistorian_AuditTrail):\n#for i in range(rowStartValidData, 1000):    # temporary\n    strg = df_DataHistorian_AuditTrail.loc[i]\n    #print('strg:', strg)\n    #dataFrame2.loc[i, 'Time'] = strg[0] \n    dataFrame2.loc[i, 'System ID'] = strg[1]\n    #print(df_DataHistorian_AuditTrail['Occurred'][i])\n    #print(df_DataHistorian_AuditTrail['Time'][i])    \n\n    datetimeObj = strg[0]\n    dataFrame2.loc[i-rowStartValidData, 'TimeInUnixTime'] = datetimeObj.timestamp()\n    dataFrame2.loc[i-rowStartValidData, 'TimeString'] = datetimeObj\n    #print('i-rowStartValidData = ', i-rowStartValidData)\n\n#dataFrame2.loc[920:1000] \n#print('---------------------')\n#dataFrame2.iloc[100]\n\n"

In [14]:
def foundMatch(record01, record02, durationAllowed=3600): # 1800 seconds
    ID01 = record01['System ID']
    time01_1 = record01['start_time']
    time01_2 = record01['end_time']
    if((not pd.isna(time01_1)) and (not pd.isna(time01_2))):
        time01 = (time01_1 + time01_2)/2

    ID02 = record02['System ID']
    time02 = record02['TimeInUnixTime']

    #print('record01:', record01)
    #print('record02:', record02)

    match = 0
    if((not pd.isna(ID01)) and (not pd.isna(ID02))):
        if (((ID01 in ID02) or (ID02 in ID01)) and abs(time01-time02)<durationAllowed):
            match = 1;

    return match

In [15]:
# compare the two dataFrames
nrows_DataHistorian_AuditTrail_valid = dataFrame2.shape[0]
for i in range(nrows_DataHistorian_ServiceNow):
    row_DataHistorian_ServiceNow = dataFrame1.iloc[i, :]
    for j in range(30350, nrows_DataHistorian_AuditTrail):  
    #for j in range(nrows_DataHistorian_AuditTrail_valid):    # temporary    
        row_DataHistorian_AuditTrail = dataFrame2.iloc[j, :]

        # average the two timestamps in AuditTrail
        match = foundMatch(row_DataHistorian_ServiceNow, row_DataHistorian_AuditTrail)

        if match == 1:
            print('ServiceNow record:', i)
            print(df_DataHistorian_ServiceNow['assigned_to'][i], '\t', df_DataHistorian_ServiceNow['work_start'][i], '\t', df_DataHistorian_ServiceNow['work_end'][i])
            print('AuditTrail record:', j)
            print(dataFrame2['System ID'][j], '\t', '\t', dataFrame2['TimeString'][j])
            print('match.')
            print('----------------------------------------------------------')

ServiceNow record: 172
James William Dickason (L008896) 	 15-09-2021 09:06 	 15-09-2021 09:44
AuditTrail record: 30357
L008896-DS 	 	 2021-09-15 09:00:00
match.
----------------------------------------------------------
ServiceNow record: 172
James William Dickason (L008896) 	 15-09-2021 09:06 	 15-09-2021 09:44
AuditTrail record: 30358
L008896-DS 	 	 2021-09-15 09:00:00
match.
----------------------------------------------------------
ServiceNow record: 172
James William Dickason (L008896) 	 15-09-2021 09:06 	 15-09-2021 09:44
AuditTrail record: 30359
L008896-DS 	 	 2021-09-15 09:00:00
match.
----------------------------------------------------------
ServiceNow record: 172
James William Dickason (L008896) 	 15-09-2021 09:06 	 15-09-2021 09:44
AuditTrail record: 30360
L008896-DS 	 	 2021-09-15 09:00:00
match.
----------------------------------------------------------
ServiceNow record: 172
James William Dickason (L008896) 	 15-09-2021 09:06 	 15-09-2021 09:44
AuditTrail record: 30361
L